In [1]:
import time
import keyboard
import k3d

In [2]:
from sympy import *
from sympy.algebras import Quaternion
def mateq2list(lhs: Matrix, rhs: Matrix):
    L = []
    for i in range(lhs.rows):
        L.append(Eq(lhs[i], rhs[i], evaluate=False))
    return L
## Variables
t = symbols("t")                                                # Tiempo
g = symbols("g")                                                # Constante gravitacional
m = symbols("m")                                                # Masa del cohete
Cd, Cl = symbols("C_d, C_l")                                    # Coeficientes aerodinámicos
Ad, Al = symbols("A_d, A_l")                                    # Areas para fuerzas aerodinámicas
rho = symbols("rho")                                            # Densidad del aire
Dx, Dy, Dz = symbols("D_x, D_y, D_z", cls=Function)             # Fuerzas aerodinámicas
Lx, Ly, Lz = symbols("L_x, L_y, L_z", cls=Function)
T = symbols("T", cls=Function)                                  # Empuje de propulsor
T_a1, T_a2 = symbols("T_theta, T_phi", cls=Function)            # Orientación de propulsor
q0, q1, q2, q3 = symbols("q0, q1, q2, q3", cls=Function)        # Orientación del cohete
vx, vy, vz = symbols("v_x, v_y, v_z", cls=Function)             # Velocidad

# Vector velocidad
v = Matrix([vx(t), vy(t), vz(t)])
# Vector dirección
h = Matrix(Quaternion.rotate_point((1, 0, 0), Quaternion(q0(t), q1(t), q2(t), q3(t))))

## Ecuaciones del movimiento en traslación
# Fuerzas Body frame
F_B = Matrix([
    T(t)*cos(T_a1(t)),
    T(t)*sin(T_a1(t))*cos(T_a2(t)),
    T(t)*sin(T_a1(t))*sin(T_a2(t))
])

# Matriz transformación de Body frame a Inertial frame
Q_BI = Matrix([
    [2*q0(t)**2+2*q1(t)**2-1, 2*(q1(t)*q2(t)-q0(t)*q3(t)), 2*(q1(t)*q3(t)+q0(t)*q2(t))],
    [2*(q1(t)*q2(t)+q0(t)*q3(t)), 2*q0(t)**2+q2(t)**2-1, 2*(q2(t)*q3(t)-q0(t)*q1(t))],
    [2*(q1(t)*q3(t)-q0(t)*q2(t)), 2*(q2(t)*q3(t)+q0(t)*q1(t)), 2*q0(t)**2+q3(t)**2-1]
])

# Ecuaciones fuerzas aerodinámicas
Fd = Cd*(rho*v.dot(v)/2*Ad*-v.normalized())
Fl = Cl*(rho*v.dot(v)/2*Ad*(v.cross(h).cross(v).normalized()))

# Segunda ley de newton
G = Matrix([0, 0, -m*g])
D = Matrix([Dx(t), Dy(t), Dz(t)])
L = Matrix([Lx(t), Ly(t), Lz(t)])
eqs_1 = mateq2list(diff(v, t), (G + D + L + Q_BI*F_B)/m)
for eq in eqs_1: display(eq)

Eq(Derivative(v_x(t), t), ((2*q0(t)*q2(t) + 2*q1(t)*q3(t))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + (-2*q0(t)*q3(t) + 2*q1(t)*q2(t))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (2*q0(t)**2 + 2*q1(t)**2 - 1)*T(t)*cos(T_theta(t)) + D_x(t) + L_x(t))/m)

Eq(Derivative(v_y(t), t), ((-2*q0(t)*q1(t) + 2*q2(t)*q3(t))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + (2*q0(t)*q3(t) + 2*q1(t)*q2(t))*T(t)*cos(T_theta(t)) + (2*q0(t)**2 + q2(t)**2 - 1)*T(t)*sin(T_theta(t))*cos(T_phi(t)) + D_y(t) + L_y(t))/m)

Eq(Derivative(v_z(t), t), (-g*m + (2*q0(t)*q1(t) + 2*q2(t)*q3(t))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (-2*q0(t)*q2(t) + 2*q1(t)*q3(t))*T(t)*cos(T_theta(t)) + (2*q0(t)**2 + q3(t)**2 - 1)*T(t)*sin(T_phi(t))*sin(T_theta(t)) + D_z(t) + L_z(t))/m)

In [3]:
## Variables
l = symbols("l")                                                # Largo del cohete
Ix, Iy, Iz = symbols("I_x, I_y, I_z")                           # Momentos de inercia principales
wx, wy, wz = symbols("omega_x, omega_y, omega_z", cls=Function) # Velocidad angular Body frame
rt, rp = symbols("r_t, r_p")                                    # Posición de propulsor y centro de presión

## Ecuaciones del movimiento en rotación
# Momentos Body frame
T_R = Matrix([rt, 0, 0])  
P_R = Matrix([rp, 0, 0])
M_B = T_R.cross(F_B) + P_R.cross(L) + P_R.cross(D)

# Ecuaciónes de euler para rotación Body frame
I = diag(Ix, Iy, Iz)
w = Matrix([wx(t), wy(t), wz(t)])
eqs_2 = mateq2list(diff(w, t), I**-1*(M_B-w.cross(I*w)))
for eq in eqs_2: display(eq)

# Convertir velocidad angular Body frame a Inertial frame
Q = Matrix([q0(t), q1(t), q2(t), q3(t)])
S = 1/2*Matrix([
    [-q1(t), -q2(t), -q3(t)],
    [q0(t), -q3(t), q2(t)],
    [q3(t), q0(t), -q1(t)],
    [-q2(t), q1(t), q0(t)]
])
eqs_3 = mateq2list(diff(Q, t), S*w)
for eq in eqs_3: display(eq)

Eq(Derivative(omega_x(t), t), (I_y*omega_y(t)*omega_z(t) - I_z*omega_y(t)*omega_z(t))/I_x)

Eq(Derivative(omega_y(t), t), (-I_x*omega_x(t)*omega_z(t) + I_z*omega_x(t)*omega_z(t) - r_p*D_z(t) - r_p*L_z(t) - r_t*T(t)*sin(T_phi(t))*sin(T_theta(t)))/I_y)

Eq(Derivative(omega_z(t), t), (I_x*omega_x(t)*omega_y(t) - I_y*omega_x(t)*omega_y(t) + r_p*D_y(t) + r_p*L_y(t) + r_t*T(t)*sin(T_theta(t))*cos(T_phi(t)))/I_z)

Eq(Derivative(q0(t), t), -0.5*omega_x(t)*q1(t) - 0.5*omega_y(t)*q2(t) - 0.5*omega_z(t)*q3(t))

Eq(Derivative(q1(t), t), 0.5*omega_x(t)*q0(t) - 0.5*omega_y(t)*q3(t) + 0.5*omega_z(t)*q2(t))

Eq(Derivative(q2(t), t), 0.5*omega_x(t)*q3(t) + 0.5*omega_y(t)*q0(t) - 0.5*omega_z(t)*q1(t))

Eq(Derivative(q3(t), t), -0.5*omega_x(t)*q2(t) + 0.5*omega_y(t)*q1(t) + 0.5*omega_z(t)*q0(t))

In [4]:
## Variables
x, y, z = symbols("x, y, z", cls=Function)  # Posición

## Sistema de ecuaciones completo
R = Matrix([x(t), y(t), z(t)])
eqs_4 = mateq2list(diff(R, t), v)
eqs = eqs_4 + eqs_1 + eqs_2 + eqs_3
eqs_rhs = []
for eq in eqs:
    display(eq)
    eqs_rhs.append(eq.rhs)

Eq(Derivative(x(t), t), v_x(t))

Eq(Derivative(y(t), t), v_y(t))

Eq(Derivative(z(t), t), v_z(t))

Eq(Derivative(v_x(t), t), ((2*q0(t)*q2(t) + 2*q1(t)*q3(t))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + (-2*q0(t)*q3(t) + 2*q1(t)*q2(t))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (2*q0(t)**2 + 2*q1(t)**2 - 1)*T(t)*cos(T_theta(t)) + D_x(t) + L_x(t))/m)

Eq(Derivative(v_y(t), t), ((-2*q0(t)*q1(t) + 2*q2(t)*q3(t))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + (2*q0(t)*q3(t) + 2*q1(t)*q2(t))*T(t)*cos(T_theta(t)) + (2*q0(t)**2 + q2(t)**2 - 1)*T(t)*sin(T_theta(t))*cos(T_phi(t)) + D_y(t) + L_y(t))/m)

...

Eq(Derivative(omega_z(t), t), (I_x*omega_x(t)*omega_y(t) - I_y*omega_x(t)*omega_y(t) + r_p*D_y(t) + r_p*L_y(t) + r_t*T(t)*sin(T_theta(t))*cos(T_phi(t)))/I_z)

Eq(Derivative(q0(t), t), -0.5*omega_x(t)*q1(t) - 0.5*omega_y(t)*q2(t) - 0.5*omega_z(t)*q3(t))

Eq(Derivative(q1(t), t), 0.5*omega_x(t)*q0(t) - 0.5*omega_y(t)*q3(t) + 0.5*omega_z(t)*q2(t))

Eq(Derivative(q2(t), t), 0.5*omega_x(t)*q3(t) + 0.5*omega_y(t)*q0(t) - 0.5*omega_z(t)*q1(t))

Eq(Derivative(q3(t), t), -0.5*omega_x(t)*q2(t) + 0.5*omega_y(t)*q1(t) + 0.5*omega_z(t)*q0(t))

In [5]:
## Sistema de ecuaciones evaluable numéricamente
sym_ss = [x(t), y(t), z(t),
          vx(t), vy(t), vz(t),
          wx(t), wy(t), wz(t),
          q0(t), q1(t), q2(t), q3(t)]
sym_pam = [m, g, l, Ix, Iy, Iz, Cd, Cl, Ad, Al, rho, rt, rp]
sym_t = [T(t), T_a1(t), T_a2(t)]
sym_d = [Dx(t), Dy(t), Dz(t)]
sym_l = [Lx(t), Ly(t), Lz(t)]

drag = lambdify((sym_ss, sym_pam), Fd)
lift = lambdify((sym_ss, sym_pam), Fl)
sys  = lambdify((sym_ss, sym_t, sym_pam, sym_d, sym_l), eqs_rhs)

In [6]:
from numpy import *
import quaternion
#https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles
def euler2quat(yaw, pitch, roll):
    cr = cos(roll * 0.5)
    sr = sin(roll * 0.5)
    cp = cos(pitch * 0.5)
    sp = sin(pitch * 0.5)
    cy = cos(yaw * 0.5)
    sy = sin(yaw * 0.5)

    q0 = cr * cp * cy + sr * sp * sy
    q1 = sr * cp * cy - cr * sp * sy
    q2 = cr * sp * cy + sr * cp * sy
    q3 = cr * cp * sy - sr * sp * cy

    return q0, q1, q2, q3


In [9]:
# Vector estado
yaw = 0
pitch = 0
roll = 0
qw, qx, qy, qz = euler2quat(yaw, pitch, roll)
ss = array([0, 0, 0,            # Posición
            0, 0, 0,            # Velocidad
            0, 0, 0,            # Velocidad angular
            qw, qx, qy, qz])
# Vector control de propulsor
tt = array([0,                  # Empuje [N]
            0, 0])              # Orientación [rad]

# Vector parametros
pam = array([10,                # Masa [kg]
             9.81,              # Aceleración de gravedad [m/s2]
             1,                 # Largo cohete [m]
             0.3, 0.3, 0.3,     # Inercias principales
             0.03, 0.03,        # Coeficientes de arrastre y lift
             0.01, 0.01,        # Areas de arrastre y lift
             1.2754,            # Densidad del aire [kg/m3]
             -1/2,              # Posición propulsors
             -1/4])             # Posición centro de presión

## Dibujar
plot = k3d.plot()
frog = k3d.stl(
    open("rocket-real-fixed.stl", "rb").read(),
    color=0x00ff00,
    transform=k3d.transform(
        scaling=[0.01,0.01,0.01],
    )
)
plot += frog
plot.display()

def update_plot():
    ## Actualizar cohete
    # La libreria requiere el cuaternion en forma Angle-Axis o sino no lo rota bien
    q = quaternion.quaternion(*ss[9:13])
    ax = quaternion.as_rotation_vector(q)   # Eje de rotación
    n = linalg.norm(ax)                     # Radianes a rotar
    frog.transform.rotation = concatenate(([n], ax))
    r = ss[0:3]
    frog.transform.translation = r
    ## Actualizar 
    plot.camera_reset()
    plot.grid = concatenate((r-ones(3), r+ones(3)))
    

## Metodo de euler
h = 1/30
while True:
    if keyboard.is_pressed("esc"): break
    dragf = drag(ss, pam).transpose()[0] if ss[3:6].any() else [0, 0, 0]
    liftf = lift(ss, pam).transpose()[0] if ss[3:6].any() else [0, 0, 0]
    ss = ss + [Y*h for Y in sys(ss, tt, pam, dragf, liftf)]
    update_plot()
    time.sleep(h)

Output()

In [ ]:
ones(3)*10

array([10., 10., 10.])